In [ ]:
import json


gt_all = []
gt_trigger = []
with open("processed_data/ace/qwen/test.jsonl", "r") as f:
    for line in f:
        events = json.loads(json.loads(line)['response'])['events']
        e_gt = []
        triggers = []
        for trigger in events:
            e_gt.append(str((trigger['trigger_text'], [(arg['text'], arg['role']) for arg in trigger['arguments']])))
            triggers.append(str((trigger['trigger_text'], trigger['type'])))
        gt_all.append(e_gt)
        gt_trigger.append(triggers)

In [ ]:
pred = []
pred_trigger = []
with open("eval_outputs/VoCuc/Qwen3-4B-ace-sft/ace_answers.jsonl", "r") as f:
    for line in f:
        events = json.loads(json.loads(line)['text'])['events']
        e_pred = []
        triggers = []
        for trigger in events:
            e_pred.append(str((trigger['trigger_text'], [(arg['text'], arg['role']) for arg in trigger['arguments']])))
            triggers.append(str((trigger['trigger_text'], trigger['type'])))
        pred.append(e_pred)
        pred_trigger.append(triggers)

In [81]:
pred[3]

["('previous', [('Barriers Bank', 'Entity'), ('BZW', 'Entity'), ('Kleinwort Benson', 'Entity'), ('McCarthy', 'Entity')])"]

In [82]:
pred_trigger[3]

['previous']

In [ ]:
from sentence_transformers import SentenceTransformer

# Download from the 🤗 Hub
model = SentenceTransformer("google/embeddinggemma-300m")


(768,) (4, 768)
tensor([[0.3008, 0.6361, 0.4927, 0.4889]])


In [84]:
acc = 0
threshold = 0.88
all_similarities = []

for p, gt in zip(pred, gt_all):
    try:
        p_embeddings = model.encode(p)
        gt_embeddings = model.encode(gt)
        similarities = model.similarity(p_embeddings, gt_embeddings)
        all_similarities.append(similarities)
        acc += (similarities.max(-1).values > threshold).sum()
    except:
        pass

In [85]:
print("precision: ", acc / sum(len(sublist) for sublist in pred))
print("recall: ", acc / sum(len(sublist) for sublist in gt_all))

precision:  tensor(0.4818)
recall:  tensor(0.4591)


In [86]:
acc = 0
threshold = 0.88
all_similarities = []

for p, gt in zip(pred_trigger, gt_trigger):
    try:
        p_embeddings = model.encode(p)
        gt_embeddings = model.encode(gt)
        similarities = model.similarity(p_embeddings, gt_embeddings)
        all_similarities.append(similarities)
        acc += (similarities.max(-1).values > threshold).sum()
    except:
        pass

In [87]:
print("precision: ", acc / sum(len(sublist) for sublist in pred))
print("recall: ", acc / sum(len(sublist) for sublist in gt_all))

precision:  tensor(0.7578)
recall:  tensor(0.7221)
